In [ ]:
import pandas as pd
from tkinter import filedialog
from tkinter import Tk
from ciso8601 import parse_datetime as to_date
import requests
from bs4 import BeautifulSoup
from time import time, sleep
from openpyxl import Workbook
import os

In [ ]:
def get_lines():
    lines = []
    root = Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    for file in filedialog.askopenfilenames():
        with open(file) as txt:
            for line in txt.readlines():
                yield line

In [ ]:
columns = [str(i) in range(1065,1077)]

os.system("cls")

len_ramos = len(ramos)
index = [[7]*len_ramos+[8]*len_ramos+[9]*len_ramos+[10]*len_ramos,ramos*4]

df_ppng = pd.DataFrame(index=index,columns=columns)
df_ppng.fillna(0.0,inplace = True)

df_rvne = pd.DataFrame(index=index,columns=columns)
df_rvne.fillna(0.0,inplace = True)

In [ ]:
def run():
    total = 0
    print("\nSelecione os arquivos: \n")
    for line in get_lines():
        tpmoid = int(line[23:27])                          # TPMOID          #
        cmpid = line[27:31]                                # CMPID           #
        ramo = line[31:35]                                 # RAMCODIGO       #
        inicio_ro = to_date(line[35:43])                   # ESPDATAINICIORO #
        fim_ro = to_date(line[43:51])                      # ESPDATAFIMRO    #
        emiss_ro = to_date(line[51:59])                    # ESPDATAEMISSRO  #
        inicio_rd = to_date(line[72:80])                   # ESPDATAINICIORD #
        fim_rd = to_date(line[80:88])                      # ESPDATAFIMRD    #
        emiss_rd = to_date(line[88:96])                    # ESPDATAEMISSRD  #
        valor_ro = float(line[59:72].replace(",", "."))    # ESPVALORMOVRO   #
        valor_rd = float(line[96:109].replace(",", "."))   # ESPVALORMOVRD   #

        valor = 0

        if emiss_rd <= data_calc < inicio_rd:
            valor = valor_rd 
        elif inicio_rd <= data_calc < fim_rd:
            valor = valor_rd*((fim_rd-data_calc).days/((fim_rd-inicio_rd).days+1))
            
        df_ppng.loc[(tpmoid, ramo),cmpid] += valor 
        
        if data_calc < emiss_rd:
            df_rvne.loc[(tpmoid, ramo),cmpid] += valor
            
        total += 1
    return total

In [ ]:
data_calc = to_date(input("\nDigite a data de cálculo: (Formato aaaammdd) \n"))
start = time()
linhas = run()
end = time()
os.system("cls")
sleep(2)
print(f"\nVelocidade: {round(end-start,2)}s\nMédia: {round(linhas/(end-start),2)} linha/s\n")

In [ ]:
df_rve = df_ppng - df_rvne

In [ ]:
"""
print("Exportando para Excel...\n\n")
with pd.ExcelWriter('Provisões.xlsx') as writer:  # doctest: +SKIP
    df_ppng.to_excel(writer, sheet_name='PPNG')
    df_rvne.to_excel(writer, sheet_name='RVNE')
    df_rve.to_excel(writer, sheet_name='RVE')
print("Sucesso!")
sleep(2)
quit()
"""